## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
os.environ["ELEVEN_API_KEY"] = os.getenv("ELEVEN_API_KEY")
os.environ["AZURE_COGS_KEY"] = os.getenv("AZURE_COGS_KEY")
os.environ["AZURE_COGS_ENDPOINT"] = os.getenv("AZURE_COGS_ENDPOINT")
os.environ["AZURE_COGS_REGION"] = os.getenv("AZURE_COGS_REGION")
os.environ["DASHSCOPE_API_KEY"] = os.getenv("DASHSCOPE_API_KEY")
os.environ["ZHIPUAI_API_KEY"] = os.getenv("ZHIPUAI_API_KEY")
zhipukey =  os.getenv("ZHIPUAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["MOONSHOT_API_KEY"] = os.getenv("MoonShot_key")
MoonShot_key = os.getenv("MoonShot_key")
os.environ["BAICHUAN_API_KEY"] = os.getenv("BAICHUAN_API_KEY")
baichuankey = os.getenv("BAICHUAN_API_KEY")

### LangGraph config
****

In [5]:
from langgraph.graph import StateGraph, END
from langchain_community.chat_models import ChatZhipuAI
from langchain.chat_models import  ChatOpenAI
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, HumanMessage


model1 = ChatOpenAI(model="gpt-4o")

model2= ChatZhipuAI(
    model="GLM-4-Plus",
    temperature=0
)
model3 = ChatOpenAI(
    openai_api_key=MoonShot_key,
    openai_api_base="https://api.moonshot.cn/v1/", 
    model_name="moonshot-v1-8k",
)

models = {
    "openai": model1,
    "moonshot": model3,
    "zhipu": model2,
}

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

#动态召唤LLM
def _call_model(state, config):
    m = models[config["configurable"].get("model", "openai")]
    response = m.invoke(state["messages"])
    return {"messages": [response]}

# Define a new graph
workflow = StateGraph(AgentState)
workflow.add_node("model", _call_model)
workflow.set_entry_point("model")
workflow.add_edge("model", END)

app = workflow.compile()

#可以这样直接呼叫，默认openai
app.invoke({"messages": [HumanMessage(content="hi")]})


{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_2f406b9113', 'finish_reason': 'stop', 'logprobs': None}, id='run-c68bae09-cd4c-42d3-a0bd-11ab9c145551-0')]}

In [ ]:
#也可以带参数呼叫
config = {"configurable": {"model": "moonshot"}}
app.invoke({"messages": [HumanMessage(content="hi")]}, config=config)